<a href="https://colab.research.google.com/github/Swathi014/End-to-End-AI-Voice-Assistance-Pipeline/blob/update/Main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install git+https://github.com/openai/whisper.git


  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-t_oq15ta
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-t_oq15ta
  Resolved https://github.com/openai/whisper.git to commit ba3f3cd54b0e5b8ce1ab3de13e32122d0d5f98ab
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)


In [3]:
!pip install pyttsx3


In [9]:
!apt-get update
!apt-get install portaudio19-dev python3-dev
!pip install pyaudio

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Ign:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Get:7 https://r2u.stat.illinois.edu/ubuntu jammy Release [5,713 B]
Get:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease [18.1 kB]
Get:9 https://r2u.stat.illinois.edu/ubuntu jammy Release.gpg [793 B]
Hit:10 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:11 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:12 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,135 kB]
Get:13 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,424 kB]
Hit:14 htt

In [3]:
!pip install SpeechRecognition


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.8/32.8 MB 58.0 MB/s eta 0:00:00


In [5]:
!pip install pydub


In [21]:
!pip install pydub webrtcvad

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.2/66.2 kB 5.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for webrtcvad: filename=webrtcvad-2.0.10-cp310-cp310-linux_x86_64.whl size=73468 sha256=47e4c50a6ab44409e5e49b61551ce4e8653126a2cbe5d5678282b19e46166d05
  Stored in directory: /root/.cache/pip/wheels/2a/2b/84/ac7bacfe8c68a87c1ee3dd3c66818a54c71599abf308e8eb35
Successfully built webrtcvad


In [1]:
!apt-get install espeak # Install the espeak text-to-speech engine

import whisper
import pyttsx3
import numpy as np
import soundfile as sf
import io
import base64
from IPython.display import Javascript
from google.colab import output
from pydub import AudioSegment
import time
from pydub import AudioSegment
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
espeak is already the newest version (1.48.15+dfsg-3).
0 upgraded, 0 newly installed, 0 to remove and 52 not upgraded.


In [ ]:
!pip install git+https://github.com/openai/whisper.git
!pip install pydub webrtcvad
!pip install transformers
!pip install bitsandbytes

In [13]:
!pip install bitsandbytes --upgrade

In [21]:
import whisper
import webrtcvad
from pydub import AudioSegment
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from transformers import BitsAndBytesConfig
import bitsandbytes as bnb
import torch

# Load Whisper model
model = whisper.load_model("base")

# Load and preprocess audio
audio = AudioSegment.from_file("/content/drive/MyDrive/Ass/record_out.wav")
audio = audio.set_frame_rate(16000).set_channels(1)
audio.export("processed_audio.wav", format="wav")

# Initialize VAD
vad = webrtcvad.Vad()
vad.set_mode(3)

# ... (Process audio frames from "processed_audio.wav" and apply VAD)
# ... (For simplicity, I'm skipping the VAD implementation here.
# ...  Refer to WebRTC VAD documentation for how to use it.)

# Transcribe non-silent audio
result = model.transcribe("processed_audio.wav", fp16=False)
text = result['text']
print(text)

# Load LLM model and tokenizer
model_name = "google/flan-t5-small"  # Use the smaller Flan-T5 model
tokenizer = AutoTokenizer.from_pretrained(model_name)
# Configure quantization
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_use_double_quant=True,
)

model = AutoModelForSeq2SeqLM.from_pretrained(
    model_name,
    device_map='auto',
    quantization_config=quantization_config
)

# Tokenize input text and add prompt for better response
prompt = f"### Instruction: Answer the following question.\n### Input: {text}\n### Response:"
inputs = tokenizer(prompt, return_tensors="pt").to('cuda')

# Generate response from LLM
# (Note: You might need to adjust generation parameters based on the model)
outputs = model.generate(**inputs, max_new_tokens=100)

# Decode and print LLM response
response = tokenizer.decode(outputs[0], skip_special_tokens=True,top_p=0.9)
print(response)

 Hi, how are you doing?
Hi, how are you doing?


In [7]:
!pip install edge-tts

In [22]:
from edge_tts import Communicate

# Use the 'response' variable as the text input
print(response)
communicator = Communicate(response)

# Generate speech and save it to a file
await communicator.save("output_speech.mp3")

Hi, how are you doing?
